In [1]:
%matplotlib inline

In [3]:
#export
from exp.nb_04 import *

exp.nb_04.CNNModel

In [ ]:
class TeacherWithAnnealing():
    
    def __init__(self, 
                 lossFunction=Functional.cross_entropy, 
                 accuracyFunction=accuracy
                ):
        self.lossFunction = lossFunction
        self.accuracyFunction = accuracyFunction
        
    def teachModel(self, cnnModel, dataBunch, learningRate, numberOfEpochs):
        self.optimizer = optim.SGD(cnnModel.parameters(), learningRate)
        for epoch in range(numberOfEpochs):
            trainingLoss, trainingAccuracy = self._trainModel(cnnModel, dataBunch.trainingDataLoader)
            print("Epoch #{} Training: Loss {} Accuracy {}".format(epoch, trainingLoss, trainingAccuracy))
            
            validationLoss, validationAccuracy = self._validateModel(cnnModel, dataBunch.validationDataLoader)
            print("Epoch #{} Validation: Loss {} Accuracy {}".format(epoch, validationLoss, validationAccuracy))
            print("")
            
    def _trainModel(self, cnnModel, trainingDataSet):
        def _teachModel(loss):
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
        returnItems = self._proccessDataSet(cnnModel, trainingDataSet, _teachModel)
        return returnItems
        
    def _validateModel(self, cnnModel, validationDataSet):
        with torch.no_grad():
            returnItems = self._proccessDataSet(cnnModel, validationDataSet)
        return returnItems
    
    def _proccessDataSet(self, 
                         cnnModel, 
                         dataLoader, 
                         postEvaluation=lambda loss: None):
        accumulatedLoss, accumulatedAccuracy = 0.,0.
        for _xDataSet, _yDataSet in dataLoader:
            _predictions = cnnModel(_xDataSet)
            loss = self.lossFunction(_predictions, _yDataSet)
            postEvaluation(loss)
            accumulatedLoss+= loss
            accumulatedAccuracy += self.accuracyFunction(_predictions, _yDataSet)
        numberOfBatches = len(dataLoader)
        return accumulatedLoss/numberOfBatches, accumulatedAccuracy/numberOfBatches
